In [1]:
#query2 DATAFRAME

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3035,application_1732639283265_2993,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import year, to_date
from pyspark.sql.functions import substring
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import count, when, length
import time

from pyspark.sql.functions import count, when, col
from pyspark.sql import Window
from pyspark.sql.functions import col, row_number, round

spark = SparkSession \
    .builder \
    .appName("DF query 2 execution") \
    .getOrCreate()

# Access configuration
conf = spark.sparkContext.getConf()

# Print relevant executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: None
Executor Memory: 4743M
Executor Cores: 2

In [3]:
conf = spark.sparkContext.getConf()

# Print relevant executor settings
print("Executor Instances:", conf.get("spark.executor.instances"))
print("Executor Memory:", conf.get("spark.executor.memory"))
print("Executor Cores:", conf.get("spark.executor.cores"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Executor Instances: None
Executor Memory: 4743M
Executor Cores: 2

In [4]:
victim19 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv",
    header=True
)

victim20 = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv",
    header=True
)

victim_df19 = victim19.select("AREA ", "Status", "DATE OCC")

victim_df20 = victim20.select("AREA", "Status", "DATE OCC")
victim_df20 = victim_df20.withColumnRenamed("AREA", "AREA ")


# Combine the two datasets
victim_df = victim_df19.union(victim_df20)

joined_data = victim_df

#joined_data.show(10)

police_station = spark.read.csv(
    "s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv",
    header=True
)

#police_station.show(3)
joined_data = joined_data.withColumn(
    "AREA ",
    when(col("AREA ") == "01", 1)
    .when(col("AREA ") == "02", 2)
    .when(col("AREA ") == "03", 3)
    .when(col("AREA ") == "04", 4)
    .when(col("AREA ") == "05", 5)
    .when(col("AREA ") == "06", 6)
    .when(col("AREA ") == "07", 7)
    .when(col("AREA ") == "08", 8)
    .when(col("AREA ") == "09", 9)
    .when(col("AREA ") == "10", 10)
    .when(col("AREA ") == "11", 11)
    .when(col("AREA ") == "12", 12)
    .when(col("AREA ") == "13", 13)
    .when(col("AREA ") == "14", 14)
    .when(col("AREA ") == "15", 15)
    .when(col("AREA ") == "16", 16)
    .when(col("AREA ") == "17", 17)
    .when(col("AREA ") == "18", 18)
    .when(col("AREA ") == "19", 19)
    .when(col("AREA ") == "20", 20)
    .when(col("AREA ") == "21", 21)
)
police_station = police_station.withColumn(
    "PREC",
    when(col("PREC") == "1", 1)
    .when(col("PREC") == "2", 2)
    .when(col("PREC") == "3", 3)
    .when(col("PREC") == "4", 4)
    .when(col("PREC") == "5", 5)
    .when(col("PREC") == "6", 6)
    .when(col("PREC") == "7", 7)
    .when(col("PREC") == "8", 8)
    .when(col("PREC") == "9", 9)
    .when(col("PREC") == "10", 10)
    .when(col("PREC") == "11", 11)
    .when(col("PREC") == "12", 12)
    .when(col("PREC") == "13", 13)
    .when(col("PREC") == "14", 14)
    .when(col("PREC") == "15", 15)
    .when(col("PREC") == "16", 16)
    .when(col("PREC") == "17", 17)
    .when(col("PREC") == "18", 18)
    .when(col("PREC") == "19", 19)
    .when(col("PREC") == "20", 20)
    .when(col("PREC") == "21", 21)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# df1.join(df2, df1['id'] == df2['id'], 'inner')
start_time = time.time()


joined_data = joined_data.withColumn(
    "YEAR", 
    when(col("DATE OCC").isNotNull() & (length("DATE OCC") >= 7), substring("DATE OCC", 7, 4)).otherwise(None)
)
#joined_data.show(3)

joined_data = joined_data.join(police_station, joined_data['AREA '] == police_station['PREC'], 'inner')

joined_data = joined_data.select("AREA ", "PREC", "DIVISION", "YEAR", "Status") \
                         .orderBy("AREA ")

# joined_data.show(22)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
joined_data=joined_data.select("AREA ","DIVISION", "YEAR", "Status") 
#joined_data.show(3)
result_total_cases = (
    joined_data.groupBy("DIVISION", "YEAR")  # Group by 'year' and 'area'
    .agg(count(when(col("Status") != "VAGGELIS", 1)).alias("Total cases"))).orderBy((["YEAR", "Total Cases"]), desceding=[True,True])

# Show the result
# result_total_cases.show(22)


result_open_cases = (
    joined_data.groupBy("DIVISION", "YEAR")
    .agg(
        count(
            when(
                (col("Status") == "UNK") | (col("Status") == "CC") | col("Status").isNull() | (col("Status") == "IC"), 1
            )
        ).alias("Open Cases") 
    )
    .orderBy(col("Open Cases").desc())
)

# Show the result
# result_open_cases.show(22)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# joined_data = joined_data.join(police_station, joined_data['AREA '] == police_station['PREC'], 'inner')
# df1.join(df2, (df1["column1"] == df2["column1"]) & (df1["column2"] == df2["column2"]), "join_type")


final_data = result_total_cases.join(
    result_open_cases, 
    (result_total_cases["YEAR"] == result_open_cases["YEAR"]) & (result_total_cases["DIVISION"] == result_open_cases["DIVISION"]),
    "inner"
)
final_data = final_data.drop(result_open_cases["DIVISION"]).drop(result_open_cases["YEAR"])

final_data = final_data.withColumn(
    "closed_case_rate", 
    100*(1 - (col("Open Cases") / col("Total Cases")))
)
final_data = final_data.withColumnRenamed("DIVISION", "precinct") \
                       .withColumnRenamed("YEAR", "year")
final_data = final_data.drop("Open Cases", "Total Cases")



window_spec = Window.partitionBy("year").orderBy(col("closed_case_rate").desc())

final_data = final_data.withColumn("ranking", row_number().over(window_spec))

final_data = final_data.orderBy("year", "ranking")

final_data.select("year", "precinct", "closed_case_rate", "ranking")

final_data = final_data.filter(F.col("ranking") <= 3)

final_data.show()

end_time = time.time()
elapsed_time = end_time - start_time

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----+------------------+-------+
|   precinct|year|  closed_case_rate|ranking|
+-----------+----+------------------+-------+
|    RAMPART|2010| 32.84713448949121|      1|
|    OLYMPIC|2010|31.515289821999083|      2|
|     HARBOR|2010| 29.36028339237341|      3|
|    OLYMPIC|2011|35.040060090135206|      1|
|    RAMPART|2011|  32.4964471814306|      2|
|     HARBOR|2011|28.513362463164317|      3|
|    OLYMPIC|2012| 34.29708533302118|      1|
|    RAMPART|2012| 32.46000463714353|      2|
|     HARBOR|2012|29.509585848956675|      3|
|    OLYMPIC|2013|33.582179409993984|      1|
|    RAMPART|2013|  32.1060382916053|      2|
|     HARBOR|2013|29.715302491103202|      3|
|   VAN NUYS|2014|32.021523528170505|      1|
|WEST VALLEY|2014| 31.49754809505847|      2|
|    MISSION|2014|31.224939855653567|      3|
|   VAN NUYS|2015|32.265140677157845|      1|
|    MISSION|2015| 30.46376267367631|      2|
|   FOOTHILL|2015|30.353001803658852|      3|
|   VAN NUYS|2016|  32.19451846212

In [8]:
print(f"Time taken: {elapsed_time:.2f} seconds")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Time taken: 12.13 seconds